In [2]:
# Abrir una conexión con neo4j

from neo4j import GraphDatabase

DIRECCION = "neo4j://54.75.63.152:7687"
AUTENTICACION = ("neo4j", "password")

conexion = GraphDatabase.driver( DIRECCION , auth=AUTENTICACION )

In [3]:
def ejecutar(QUERY, PARAMETROS= None):
    session = None
    RESPUESTA = None
    try:
        session = conexion.session()
        RESPUESTA = list(session.run( QUERY, PARAMETROS ))
    except Exception as e:
        print("Error al ejecutar query", e)
    finally:
        session.close()
    return RESPUESTA

In [4]:
ejecutar( 'CREATE (:Persona { nombre: "Ivan" } )' )

[]

In [7]:
resultado = ejecutar( 'MATCH (ivan:Persona { nombre: "Ivan" } ) RETURN ivan' )

In [8]:
ivan = resultado[0]
ivan

<Record ivan=<Node element_id='4:598ae28c-8332-445e-83c8-afc0ece5a1e1:0' labels=frozenset({'Persona'}) properties={'nombre': 'Ivan'}>>

In [14]:
import pandas as pd
import requests
import io 

peliculas_csv = requests.get("https://data.neo4j.com/intro/movies/movies.csv").content.decode("utf-8")

peliculas = pd.read_csv(io.StringIO(peliculas_csv)) # Nos simula que tenemos un fichero con ese contenido
peliculas.head(2)

,title,released,tagline
0,The Matrix,1999,Welcome to the Real World
1,Something's Gotta Give,1975,NaN


In [16]:
peliculas.head(2).to_dict("records")

[{'title': 'The Matrix',
  'released': 1999,
  'tagline': 'Welcome to the Real World'},
 {'title': "Something's Gotta Give", 'released': 1975, 'tagline': nan}]

In [17]:
# UNWIND itera por cada elemento de la lista. Sería el equivalente a un for_each
query_insercion_peliculas = '''
                            UNWIND $filas AS fila
                            CREATE (pelicula:Pelicula {titulo: fila.title , fecha: fila.released , eslogan: fila.tagline})
                            RETURN count(pelicula) as total
                            '''
parametros = { "filas": peliculas.to_dict("records") }
resultado = ejecutar( query_insercion_peliculas , parametros ) 

In [18]:
resultado

[<Record total=38>]

In [19]:
personas_csv = requests.get("https://data.neo4j.com/intro/movies/people.csv").content.decode("utf-8")
personas = pd.read_csv(io.StringIO(personas_csv)) # Nos simula que tenemos un fichero con ese contenido
personas.head(2)

,name,born
0,Michael Sheen,1969.0
1,Jack Nicholson,1937.0


In [22]:
query_insercion_personas = '''
                            UNWIND $filas AS fila
                            CREATE (persona:Persona {nombre: fila.name , nacimiento: fila.born})
                            RETURN count(persona) as total
                            '''
parametros = { "filas": personas.to_dict("records") }
resultado = ejecutar( query_insercion_personas , parametros ) 
resultado

[<Record total=102>]

In [21]:
directores_csv = requests.get("https://data.neo4j.com/intro/movies/directors.csv").content.decode("utf-8")
directores = pd.read_csv(io.StringIO(directores_csv)) # Nos simula que tenemos un fichero con ese contenido
directores.head(2)

,person,movie
0,Lilly Wachowski,The Matrix
1,Lana Wachowski,The Matrix


In [23]:
query_insercion_directores = '''
                            UNWIND $filas AS fila
                            MATCH (persona:Persona {nombre: fila.person})
                            MATCH (pelicula:Pelicula {titulo: fila.movie})
                            MERGE (persona)-[dirigio:Dirigio]->(pelicula)
                            RETURN count(dirigio) as total
                            '''
parametros = { "filas": directores.to_dict("records") }
resultado = ejecutar( query_insercion_directores , parametros ) 
resultado

[<Record total=6>]

In [24]:
# ON CREATE SET
actores_csv = requests.get("https://data.neo4j.com/intro/movies/actors.csv").content.decode("utf-8")
actores = pd.read_csv(io.StringIO(actores_csv)) # Nos simula que tenemos un fichero con ese contenido
actores.head(2)

,movie,roles,person
0,Something's Gotta Give,Julian Mercer,Keanu Reeves
1,Johnny Mnemonic,Johnny Mnemonic,Keanu Reeves


In [25]:
query_insercion_actores = '''
                            UNWIND $filas AS fila
                            MATCH (persona:Persona {nombre: fila.person})
                            MATCH (pelicula:Pelicula {titulo: fila.movie})
                            MERGE (persona)-[actuo:Actuo]->(pelicula)
                            ON CREATE SET actuo.papeles = split(fila.roles,";")
                            RETURN count(actuo) as total
                            '''
parametros = { "filas": actores.to_dict("records") }
resultado = ejecutar( query_insercion_actores , parametros ) 
resultado

[<Record total=172>]

In [39]:
argumentos = {"nombres": ["Keanu Reeves","Tom Cruise"]}
resultado=ejecutar("""
                    UNWIND $nombres AS nombre
                    MATCH (Persona{nombre: nombre})-[relacion]->(Pelicula)
                    <-[Actuo]-(interesado:Persona)
                    RETURN interesado.nombre, interesado.nacimiento
                    """ , argumentos)
datos=pd.DataFrame(resultado)
datos.columns=["Nombre", "Año"]

In [40]:
datos

,Nombre,Año
0,Emil Eifrem,1978.0
1,Hugo Weaving,1960.0
2,Laurence Fishburne,1961.0
3,Carrie-Anne Moss,1967.0
4,Hugo Weaving,1960.0
5,Laurence Fishburne,1961.0
6,Carrie-Anne Moss,1967.0
7,Hugo Weaving,1960.0
8,Laurence Fishburne,1961.0
9,Carrie-Anne Moss,1967.0
